In [1]:
import hashlib
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import io
import logging
import pandas as pd
import pandahouse
from read_db.CH import Getch
import os
sns.set

<function seaborn.rcmod.set(*args, **kwargs)>

In [2]:
def ab_split(id, salt='exp_mess_1', n_groups=5):
    test_id = str(id) + '-' + str(salt)
    test_id_digest = hashlib.md5(test_id.encode('ascii')).hexdigest()
    test_id_final_int = int(test_id_digest, 16)
    return test_id_final_int % n_groups

In [3]:
connection = {}

q = """
SELECT distinct user_id
FROM simulator_20220320.feed_actions 
WHERE toDate(time) >= '2022-03-15' and toDate(time) <= '2022-03-21' 
"""

# Вытащили пользователей
users_df = pandahouse.read_clickhouse(q, connection=connection)

# Сделали группы
users_df['hash_group'] = users_df.user_id.apply(ab_split)

# Смотрим на число пользователей в каждой группе
users_df.groupby('hash_group').user_id.nunique().reset_index()

,hash_group,user_id
0,0,10151
1,1,9982
2,2,9977
3,3,9963
4,4,10076


In [4]:
q = """
SELECT xxHash64(toString(user_id)||'my_salt')%5 as group, 
    uniqExact(user_id)
FROM simulator_20220320.feed_actions 
WHERE toDate(time) >= '2022-03-15' and toDate(time) <= '2022-03-21' 
group by group 
"""

pandahouse.read_clickhouse(q, connection=connection)

,group,uniqExact(user_id)
0,0,10091
1,1,9976
2,2,9966
3,3,10140
4,4,9976


In [27]:
q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views
FROM simulator_20220320.feed_actions 
WHERE toDate(time) >= '2022-03-15' and toDate(time) <= '2022-03-21' 
    and exp_group in (1)
GROUP BY exp_group, user_id
"""

df = pandahouse.read_clickhouse(q, connection=connection)

df.groupby('exp_group').count()

,user_id,likes,views
exp_group,,,
1,10079,10079,10079


In [28]:
CTRcontrol = df['likes'].sum()/df['views'].sum()
CTRcontrol

0.20802680490126244

In [36]:
q = """
SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr,
    likes-0.20802680490126244*views as linearized_likes
FROM simulator_20220320.feed_actions 
WHERE toDate(time) >= '2022-03-15' and toDate(time) <= '2022-03-21' 
    and exp_group in (0,1,2,3)
GROUP BY exp_group, user_id
"""

df = pandahouse.read_clickhouse(q, connection=connection)

df.groupby('exp_group').count()

,user_id,likes,views,ctr,linearized_likes
exp_group,,,,,
0,9999,9999,9999,9999,9999
1,10079,10079,10079,10079,10079
2,9952,9952,9952,9952,9952
3,10049,10049,10049,10049,10049


In [33]:
df.head()

,exp_group,user_id,likes,views,ctr,linearized_likes
0,3,115383,9,30,0.300000,2.759196
1,3,123580,13,48,0.270833,3.014713
2,0,4944,8,41,0.195122,-0.529099
3,0,4504,5,15,0.333333,1.879598
4,0,121508,18,88,0.204545,-0.306359


In [40]:
stats.ttest_ind(df[df.exp_group == 0].ctr,
                df[df.exp_group == 3].ctr,
                equal_var=False)

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)

In [39]:
stats.ttest_ind(df[df.exp_group == 0].linearized_likes,
                df[df.exp_group == 3].linearized_likes,
                equal_var=False)

Ttest_indResult(statistic=-15.216492682365713, pvalue=5.368589044103529e-52)

In [38]:
stats.ttest_ind(df[df.exp_group == 1].ctr,
                df[df.exp_group == 2].ctr,
                equal_var=False)

Ttest_indResult(statistic=0.7094392041270485, pvalue=0.4780623130874935)

In [41]:
stats.ttest_ind(df[df.exp_group == 1].linearized_likes,
                df[df.exp_group == 2].linearized_likes,
                equal_var=False)

Ttest_indResult(statistic=6.122579994775972, pvalue=9.439432187037712e-10)

In [ ]:
#p-value вырос в обоих случаях